In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

In [3]:
df = pd.read_csv('../../Data_Files/Raw_Data/users_ratings.csv')
df_filtered = df[['UserID', 'Title', 'User_Rating']]
df_filtered

,UserID,Title,User_Rating
0,ur117926588,A Nightmare on Elm Street 2: Freddy's Revenge,5
1,ur117926588,Elvis,8
2,ur117926588,Where Eagles Dare,9
3,ur117926588,A Nightmare on Elm Street,9
4,ur117926588,Eagle Eye,5
...,...,...,...
15590,ur4532636,Jackie Brown,10
15591,ur4532636,Beauty and the Beast,6
15592,ur4532636,Dirty Pretty Things,9
15593,ur4532636,"Easy Riders, Raging Bulls: How the Sex, Drugs ...",7


In [4]:
df_rating = df_filtered.copy()
df_rating['combined'] = df_rating['UserID'] + df_rating['Title']
counts = df_rating['combined'].value_counts()
unique_counts = counts[counts == 1]
df_rating = df_rating[df_rating['combined'].isin(unique_counts.index)]

In [5]:
pivoted_table = df_rating.pivot(index='UserID',columns='Title',values='User_Rating').fillna(0)

In [6]:
pivoted_table

Title,#Alive,'71,'G' Men,'Pimpernel' Smith,*batteries not included,...All the Marbles,10,10 Cloverfield Lane,10 Rillington Place,10 Things I Hate About You,...,Zootopia,Zorba the Greek,Zumbo's Just Desserts,[REC] 3: Genesis,[REC] 4: Apocalypse,[Rec]²,eXistenZ,iBoy,"tick, tick... BOOM!",xXx
UserID,,,,,,,,,,,,,,,,,,,,,
ur117926588,9.0,9.0,0.0,0.0,7.0,5.0,7.0,9.0,8.0,0.0,...,9.0,0.0,7.0,0.0,0.0,0.0,8.0,6.0,0.0,4.0
ur15298231,0.0,6.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
ur17646017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur1994077,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur22171966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,6.0,...,10.0,0.0,0.0,3.0,5.0,8.0,5.0,0.0,0.0,0.0
ur4532636,0.0,8.0,6.0,0.0,0.0,0.0,4.0,6.0,0.0,0.0,...,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [153]:
model = NMF(init='nndsvd', n_components=len(pivoted_table), solver='mu', l1_ratio=0.5, max_iter=500, random_state=0)
users_matrix = model.fit_transform(pivoted_table)
ratings_matrix = model.components_
predictions = np.dot(users_matrix, ratings_matrix)

c:\Users\Guo Xiong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\decomposition\_nmf.py:1524: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


In [154]:
threshold = 0.001
recommendations = abs(pivoted_table - predictions)
filtered = (recommendations[recommendations>threshold]).dropna(how='all')
filtered

Title,#Alive,'71,'G' Men,'Pimpernel' Smith,*batteries not included,...All the Marbles,10,10 Cloverfield Lane,10 Rillington Place,10 Things I Hate About You,...,Zootopia,Zorba the Greek,Zumbo's Just Desserts,[REC] 3: Genesis,[REC] 4: Apocalypse,[Rec]²,eXistenZ,iBoy,"tick, tick... BOOM!",xXx
UserID,,,,,,,,,,,,,,,,,,,,,
ur117926588,0.001997,0.002486,0.305397,NaN,0.001553,0.001109,NaN,0.068518,0.001775,0.606412,...,0.082606,0.356377,0.001553,NaN,NaN,0.001150,0.001644,0.001331,0.050848,NaN
ur15298231,NaN,NaN,NaN,NaN,NaN,NaN,0.042498,0.064963,NaN,0.025123,...,0.076151,0.070418,NaN,0.001583,0.002638,0.004220,0.002709,NaN,NaN,NaN
ur17646017,0.015924,0.052127,0.019099,0.002488,0.012385,0.008847,0.033032,0.077024,0.014155,0.232605,...,0.088568,0.017488,0.012385,NaN,0.001541,0.002465,0.016477,0.010616,0.006346,0.007077
ur1994077,0.127525,0.158942,0.027773,NaN,0.099186,0.070847,NaN,4.847525,0.113356,0.088338,...,5.843313,0.032416,0.099186,NaN,NaN,NaN,0.113374,0.085017,0.004621,0.056678
ur22171966,0.036730,0.035273,NaN,NaN,0.028568,0.020406,0.027879,NaN,0.032649,NaN,...,NaN,NaN,0.028568,NaN,NaN,NaN,NaN,0.024487,NaN,0.016325
ur4532636,0.009471,NaN,0.015745,0.001317,0.007366,0.005262,NaN,0.017943,0.008419,1.289707,...,0.021673,0.019102,0.007366,NaN,NaN,NaN,0.007510,0.006314,0.002617,0.004209


In [155]:
all_recos = {}
to_reco = 5

for index, row in filtered.iterrows():
    recos = row[row>0].sort_values(ascending=False)[:to_reco].index.tolist()
    print(row[row>0].sort_values(ascending=False)[:to_reco])
    all_recos[index] = recos

all_recos

Title
The Sound of Music     9.200346
Life of Pi             9.190400
Coraline               8.980741
Edward Scissorhands    8.930140
Lawrence of Arabia     8.879347
Name: ur117926588, dtype: float64
Title
The Man Who Wasn't There    3.898993
Gravity                     3.887474
The Seventh Seal            2.929069
Incredibles 2               2.926575
Gone Girl                   2.924639
Name: ur15298231, dtype: float64
Title
Life of Pi                                      1.705049
Amadeus                                         1.637669
Star Wars: Episode III - Revenge of the Sith    1.589923
The Sound of Music                              1.561247
Schindler's List                                1.493871
Name: ur17646017, dtype: float64
Title
The Usual Suspects       6.757139
Toy Story                6.736200
Zootopia                 5.843313
It's a Wonderful Life    5.833158
Fantasia                 5.829181
Name: ur1994077, dtype: float64
Title
Jurassic Park III    2.915305
This Is 

{'ur117926588': ['The Sound of Music',
  'Life of Pi',
  'Coraline',
  'Edward Scissorhands',
  'Lawrence of Arabia'],
 'ur15298231': ["The Man Who Wasn't There",
  'Gravity',
  'The Seventh Seal',
  'Incredibles 2',
  'Gone Girl'],
 'ur17646017': ['Life of Pi',
  'Amadeus',
  'Star Wars: Episode III - Revenge of the Sith',
  'The Sound of Music',
  "Schindler's List"],
 'ur1994077': ['The Usual Suspects',
  'Toy Story',
  'Zootopia',
  "It's a Wonderful Life",
  'Fantasia'],
 'ur22171966': ['Jurassic Park III',
  'This Is the End',
  'Paul',
  'Lucy',
  'Grown Ups'],
 'ur4532636': ['Syriana',
  'The Thin Red Line',
  '12 Angry Men',
  'Dead Man Walking',
  'The Elephant Man']}